In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  7 13:03:32 2017

@author: sergio
"""

import numpy as np
#import random
import codecs
import math
import pandas
from scipy.spatial import Voronoi#, voronoi_plot_2d
from scipy.stats import wilcoxon, ranksums
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
ORIGIN = "../static/ctc/TWITTER_COLOMBIA_RANK_NORM.csv"
RANK_FILE = "../static/ctc/TWITTER_COLOMBIA_RANK.csv"
HSIC_FILTER = "../static/ctc/HSIC_RECOLECT/CTC_HSIG_SOBRE_RANK_NORMALIZADO.csv"
COORDINATES = "../static/geo/valid_cities_ccordinates_ext.csv"

data = pandas.read_csv(COORDINATES, sep="\t", index_col=0, decimal=b',').sort_index()
hsic_filter_words = pandas.read_csv(HSIC_FILTER, sep="\t", index_col=0, decimal=b',')

In [3]:
#hsic_filter_words.sort_values(by="HSIC", ascending=False)
#data

## Isoglosas

In [4]:
#APARITION TREHS

def aparition_tresh_file(file, tresh):
    reader=pandas.read_csv(file, sep="\t", index_col=0, decimal=b',')
    aparition = reader.iloc[0:]<(reader.iloc[0]-tresh)
    aparition.to_csv("../static/ctc/aparition_{}.csv".format(tresh), sep="\t",decimal=",",header=aparition.columns)
    


#aparition_tresh_file(RANK_FILE, 0)

In [5]:
#gets Voronoi regions
points = np.array(data)
vor = Voronoi(points)
#TOTAL_WORDS_TRESH = TOTAL_WORDS["#TOTAL_WORDS#"]*APARITION_TRESH
#VALID_WORDS_TRESH = (VALID_WORDS>TOTAL_WORDS_TRESH).astype(int)
#VALID_WORDS_ABS_FREC =  VALID_WORDS/TOTAL_WORDS.values[:,0]

# Filtrado de palabras

In [6]:
source_data=pandas.read_csv(ORIGIN, sep="\t", chunksize=5000, index_col=0, decimal=b',')

i=0
for chunk in source_data:
    words_to_drop = list(set(list(chunk.index))-set(hsic_filter_words.index))
    chunk.drop(labels=words_to_drop, inplace=True)
    if i==0:
        filtredwords=chunk
    else:
        filtredwords=filtredwords.append(chunk)
    i+=1
filtredwords.to_csv("../static/ctc/filterwords.csv".format(i), sep="\t",decimal=",",header=filtredwords.columns)
#TOTAL_WORDS = pandas.DataFrame(source_data.loc["#TOTAL_WORDS#"]).drop(labels=['HSIC','PVALUE'])
#WORDS = source_data.drop(labels=['#TOTAL_WORDS#','#TWEETS#',"#VOCABULARY_SIZE#","#USERS#"])
#VALID_WORDS = WORDS.loc[(WORDS['PVALUE'] < PVALUE_TRESH)].drop(labels=['HSIC','PVALUE'], axis=1)


In [7]:
VALID_WORDS=pandas.read_csv("../static/ctc/filterwords.csv", sep="\t", index_col=0, decimal=b',') 
VALID_WORDS=VALID_WORDS.transpose().sort_index().transpose()

In [8]:
def minlength_frame(pueblo1,pueblo2):
    A=pandas.DataFrame(VALID_WORDS[pueblo1])
    A = A[A[pueblo1] != 1.000000e+08]

    B=pandas.DataFrame(VALID_WORDS[pueblo2])
    B = B[B[pueblo2] != 1.000000e+08]

    if len(A)<len(B):
        return (len(A), A.columns.values[0],A)
    elif len(A)>len(B):
        return (len(B), B.columns.values[0],B)
    else:
        C=pandas.concat([A,B], axis=1).fillna(1.000000e+08)
        return (len(C), 'Iguales',C)


def max_leng_frame(pueblo1,pueblo2):
    A=pandas.DataFrame(VALID_WORDS[pueblo1])
    A = A[A[pueblo1] != 1.000000e+08]

    B=pandas.DataFrame(VALID_WORDS[pueblo2])
    B = B[B[pueblo2] != 1.000000e+08]

    C=pandas.concat([A,B], axis=1).fillna(1.000000e+08)
    return C

In [9]:
def total_leng_frame(pueblo1,pueblo2):
    A=pandas.DataFrame(VALID_WORDS[pueblo1])
    B=pandas.DataFrame(VALID_WORDS[pueblo2])
    C=pandas.concat([A,B], axis=1)
    return (C)

def intersection_frame(pueblo1,pueblo2):
    A=pandas.DataFrame(VALID_WORDS[pueblo1])
    A = A[A[pueblo1] != 1.000000e+08]

    B=pandas.DataFrame(VALID_WORDS[pueblo2])
    B = B[B[pueblo2] != 1.000000e+08]

    C=pandas.concat([A,B], axis=1).fillna(1.000000e+08)
    C = C[C[pueblo2] != 1.000000e+08]
    C = C[C[pueblo1] != 1.000000e+08]
    return C

In [41]:
pueblo1="Medellín"
pueblo2="Santa Fe de Antioquia"

intersection_frame(pueblo1,pueblo2)["Medellín"].median()



67955801.10497236

In [10]:
def join_valid_words_minlength(pueblo1,pueblo2):
    lenx, pueblo, frame = minlength_frame(pueblo1,pueblo2)
    ind = frame.index.values
    if pueblo == pueblo1:
        B=pandas.DataFrame(VALID_WORDS[pueblo2]).filter(items=ind, axis=0)
        return(frame.join(B))
    elif pueblo == pueblo2:
        A=pandas.DataFrame(VALID_WORDS[pueblo1]).filter(items=ind, axis=0)
        return(A.join(frame))
    elif pueblo=='Iguales':
        return(frame)

In [21]:
def this_wilcoxon(pueblo1, pueblo2):
    #vectores=join_valid_words_minlength(pueblo1,pueblo2)
    vectores=max_leng_frame(pueblo1,pueblo2)
    #vectores=intersection_frame(pueblo1,pueblo2)
    n = len(vectores)
    s=n*(n+1)/2
    maxim = max(vectores[pueblo1].max(),vectores[pueblo2].max())
    minim = min(vectores[pueblo1].min(),vectores[pueblo2].min())
    diferencia = abs((vectores[pueblo1].rank(method="dense", ascending=False)-vectores[pueblo2].rank(method="dense", ascending=False)).sum())
    wil, pvalue = wilcoxon(vectores[pueblo1],vectores[pueblo2])
    peso = diferencia/(n*(maxim-minim))
    return(peso, wil, pvalue)



In [23]:
'''def constante_proporcionalidad(pueblo1, pueblo2):
    corpus=VALID_WORDS
    commun_corpus = max_leng_frame(pueblo1,pueblo2)
    commun_says =  commun_corpus[commun_corpus[pueblo1] != 1.000000e+08]
    commun_says =  commun_says[commun_says[pueblo2] != 1.000000e+08]
    not_in_smaller = len(commun_corpus.index)-len(commun_says)
    ausentes_corpus=len(corpus.index)-len(commun_corpus.index)
    k=(len(commun_says.index)+ausentes_corpus)*len(commun_corpus.index)/(((len(commun_says.index)+not_in_smaller)*(len(corpus.index))))
    return  k'''


'def constante_proporcionalidad(pueblo1, pueblo2):\n    corpus=VALID_WORDS\n    commun_corpus = max_leng_frame(pueblo1,pueblo2)\n    commun_says =  commun_corpus[commun_corpus[pueblo1] != 1.000000e+08]\n    commun_says =  commun_says[commun_says[pueblo2] != 1.000000e+08]\n    not_in_smaller = len(commun_corpus.index)-len(commun_says)\n    ausentes_corpus=len(corpus.index)-len(commun_corpus.index)\n    k=(len(commun_says.index)+ausentes_corpus)*len(commun_corpus.index)/(((len(commun_says.index)+not_in_smaller)*(len(corpus.index))))\n    return  k'

In [24]:
def weigth_values():
    weigth_values = {}    
    for i in range(len(vor.ridge_vertices)):
        vertex1,vertex2=vor.ridge_vertices[i]  # points in the map defining the line
        city1,city2=vor.ridge_points[i]  # indexes of the cities separated by the line   
        if vertex1==-1 or vertex2==-1:
            continue
        pueblo1=VALID_WORDS.iloc[:,city1].name
        pueblo2=VALID_WORDS.iloc[:,city2].name
        frontera = '{}--{}'.format(pueblo1,pueblo2)
        peso, wil, pvalue=this_wilcoxon(pueblo1,pueblo2)
       # peso = peso * constante_proporcionalidad(pueblo1, pueblo2) 
        if (pvalue>0.05):
            color=(0,1,1,1)
        if (pvalue<=0.05):
            color=(1,0,0,1)
        weigth_values[frontera]={'peso':peso,'color':color, 'orden':i}
    weigth_values=pandas.DataFrame.from_dict(weigth_values, orient='index').sort_values(by=['peso'],  ascending=False)
    maxi = weigth_values['peso'].max()
    mini = weigth_values['peso'].min()
    return (maxi, mini, weigth_values)    

In [49]:
A=[1,2,3,4,5]
B=[7,9,3.3,9,5]
wilcoxon(A,B,zero_method="zsplit")

/opt/conda/lib/python3.5/site-packages/scipy/stats/morestats.py:2388: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


WilcoxonResult(statistic=0.5, pvalue=0.059058229090536707)

In [27]:
def maping():
    plt.close('all')
    plt.clf()
    plt.figure(figsize=(80,40))

    # setup Lambert Conformal basemap.
    m = Basemap(width=1500000,height=2000000,projection='lcc',
            resolution='h',lat_1=15.,lat_2=25,lat_0=3.9,lon_0=-73.)
    # draw coastlines.
    m.drawcoastlines(linewidth=4)
    m.drawcountries(linewidth=4)
    #m.drawmapboundary(fill_color='aquamarine')
    #m.fillcontinents(color="ivory",lake_color="aquamarine")
    #m.etopo()
    m.drawstates()
    #m.shadedrelief()

    # draws locations
    for index, row in data.iterrows():
        m.tissot(row['Longitud'],row['Latitud'],radius_deg=5/200,npts=20)
        x,y=m(row['Longitud'],row['Latitud'])
        plt.text(x,y,index,fontsize=7)
   
    
    maxi,mini,weigth = weigth_values()
    weigth.reset_index(level=0, inplace=True)
    weigth.set_index('orden', inplace=True)
    weigth.sort_index(inplace=True)
    for i in range(len(vor.ridge_vertices)):
        vertex1,vertex2=vor.ridge_vertices[i]  # points in the map defining the line
        city1,city2=vor.ridge_points[i]  # indexes of the cities separated by the line
        if vertex1==-1 or vertex2==-1:
            continue
        line_weight=(weigth.loc[i]["peso"]-mini)/(maxi-mini)
        latitude1,longitude1=vor.vertices[vertex1]
        latitude2,longitude2=vor.vertices[vertex2]
        x,y=m([longitude1,longitude2],[latitude1,latitude2])    
        m.plot(x,y,color=weigth.loc[i]["color"],linewidth=10*(line_weight))
    plt.savefig('../static/ctc/prv_1.png', format='png', dpi=200)
    plt.clf()
maping()